In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import networkx as nx
from scipy import sparse

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_max_pool, GlobalAttention, GatedGraphConv
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import softmax
from torch_geometric.utils.convert import from_scipy_sparse_matrix
from torch_geometric.data import Data, DataLoader


from pyscf import gto, scf, tools, ao2mo


import train
from graph_model import SecondNet, SimpleNet, THCNet
from preprocess import build_qm7, build_thc_graph
from train import train, test
from hf import get_data, save_data, load_data, thc_to_eri_torch

Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


In [2]:
mols = build_qm7('sto-3g')
mols = mols[0:2]
filename = "sto33"

In [3]:
kwargs = {'grid_points_per_atom': 300, 'epsilon_qr': 1e-15, 'epsilon_inv': 1e-15, 'verbose': True}
save_data(mols, filename, force = True, kwargs = kwargs)
mol_data = load_data(filename)

/anaconda3/envs/prime/lib/python3.7/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:
/anaconda3/envs/prime/lib/python3.7/site-packages/pyscf/lib/misc.py:874: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  h5py.File.__init__(self, filename, *args, **kwargs)


rho L2: 1.653473217268125e-14
Sanity C as khatri-rao product: 0.0
T_ao L_infinity: 0.00017613996788602444
T_mo L_infinity: 0.00016804775254997129


In [4]:
def torch_khatri_rao(X, Y):
    C = torch.einsum("ij,kj->ikj",X,Y)
    C = C.view(X.shape[0] * Y.shape[0], -1)
    return C

In [6]:
dataset = []
for m in mol_data:
    
    X, Z, U, T_ao, T_mo, coords, mol = m.X, m.Z, m.U, m.T_ao, m.T_mo, m.coords, gto.mole.loads(m.mol)
                
    C = np.einsum("ij,kj->ikj",U.T @ X,U.T @ X).reshape((X.shape[0] ** 2, -1))
    T_approx = C @ Z @ C.T
    T_double = T_mo.reshape((X.shape[0] ** 2, X.shape[0] ** 2))
    print(np.max(np.abs(T_double - T_approx)))
    print(np.linalg.norm(T_double - T_approx))
    
#     data = build_thc_graph(m)
    data = Data(X = torch.from_numpy(X), Z = torch.from_numpy(Z),
                U = torch.from_numpy(U), coords = torch.from_numpy(coords),
                T_ao = torch.from_numpy(T_ao), T_mo = torch.from_numpy(T_mo),
               mol = mol)
    
    dataset.append(data)

0.00016804775254997129
0.0008740056166136256


In [38]:
# import math
# class PointNet(nn.Module):
#     def __init__(self, X, with_Y = False):        
#         super(PointNet, self).__init__()
        
#         self.X = nn.Parameter(torch.Tensor(X.shape[0], X.shape[1]))
#         nn.init.kaiming_uniform_(self.X, a=math.sqrt(5))
        
#         if with_Y:
#             self.Y = nn.Parameter(torch.Tensor(X.shape[0], X.shape[1]))
#             nn.init.kaiming_uniform_(self.Y, a=math.sqrt(5))
#         else:
#             self.Y = self.X
                        
#     def forward(self, mol):
#         return self.X, self.Y
    
# class GTO(torch.autograd.Function):

#     @staticmethod
#     def forward(ctx, input, mol):

#         ctx.save_for_backward(input)
#         ctx.mol = mol
#         ao_value = mol.eval_gto("GTOval_sph", input.cpu().numpy())
#         return torch.from_numpy(ao_value).T

#     @staticmethod
#     def backward(ctx, grad_output):

#         input, = ctx.saved_tensors
#         mol = ctx.mol
        
#         ao_value = mol.eval_gto("GTOval_sph", input.cpu().numpy())
        
#         ao_grad = mol.eval_gto("GTOval_ip_sph", input.cpu().numpy())
#         ao_grad = torch.from_numpy(ao_grad)
#         ao_grad = torch.transpose(ao_grad, 1, 2)
        
        
#         grad_input = torch.diagonal(grad_output.T.unsqueeze(0) @ ao_grad, dim1 = 1, dim2 = 2)

#         return grad_input.T, None
    
# phi = GTO.apply

# class SpatialPointNet(nn.Module):
#     def __init__(self, coords):        
#         super(SpatialPointNet, self).__init__()
        
#         with torch.no_grad():
#             self.coords = nn.Parameter(coords.clone())
                        
#     def forward(self, mol):
#         X = phi(self.coords, mol)
#         return X


In [40]:
import torch.optim as optim

def train(model, loader, lr = 0.003, iterations = 10, criterion = nn.MSELoss(),
          verbose = False, device = torch.device("cpu"), lamb = None, epsilon = 1e-15, grad_clip = None):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    losses = []
    for i in range(iterations):
        for data in loader:
            
            M, n = data.X.shape
            
            T_ao = data.T_ao.view(M ** 2, M ** 2)
            T_mo = data.T_mo.view(M ** 2, M ** 2)

            X_hat, Y_hat = model(data.mol)
            if lamb is not None:
                X_hat = lamb * X_hat + data.X
                Y_hat = lamb * Y_hat + data.X
            
            T_approx_fixed = thc_to_eri_torch(data.X, data.U, T_ao, epsilon)
            T_approx_learned = thc_to_eri_torch(X_hat, data.U, T_ao, epsilon, Y_hat)

            
            loss = torch.norm(T_approx_learned - T_mo)
                    
            optimizer.zero_grad()
            loss.backward()
                        
            if grad_clip is not None:
                torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            
            optimizer.step()
                        
            losses.append(loss.item())

            if verbose:
                print("timestep: {}, loss: {:e}".format(i, loss.item()))
                
            if i == iterations - 1:
                print("FINAL")
                print("loss: {:e}".format(loss.item()))
                fixed_loss = torch.norm(T_approx_fixed - T_mo).item()
                print("fixed: {:e}".format(fixed_loss))
    
    model.eval()
    return losses

In [41]:
model = PointNet(dataset[0].X).double()
lr = 0.008
lamb = 1e-4

# model = SpatialPointNet(dataset[0].coords).double()
# lr = 0.001
# lamb = None

verbose = True

In [42]:
losses = train(model, dataset, iterations = 200, lr = lr, verbose = verbose, lamb = lamb)

timestep: 0, loss: 5.024740e-03
timestep: 1, loss: 3.051993e-03
timestep: 2, loss: 1.101305e-03
timestep: 3, loss: 8.779428e-04
timestep: 4, loss: 7.249134e-04
timestep: 5, loss: 6.337369e-04
timestep: 6, loss: 5.093174e-04
timestep: 7, loss: 5.412347e-04
timestep: 8, loss: 3.089312e-04
timestep: 9, loss: 4.264008e-04
timestep: 10, loss: 3.656393e-04
timestep: 11, loss: 3.719416e-04
timestep: 12, loss: 3.323896e-04
timestep: 13, loss: 2.428713e-04
timestep: 14, loss: 2.499967e-04
timestep: 15, loss: 2.754783e-04
timestep: 16, loss: 1.990220e-04
timestep: 17, loss: 1.484193e-04
timestep: 18, loss: 2.612016e-04
timestep: 19, loss: 2.170164e-04
timestep: 20, loss: 1.399709e-04
timestep: 21, loss: 2.221091e-04
timestep: 22, loss: 2.157565e-04
timestep: 23, loss: 1.578819e-04
timestep: 24, loss: 2.337199e-04
timestep: 25, loss: 1.458551e-04
timestep: 26, loss: 2.072459e-04
timestep: 27, loss: 1.595805e-04
timestep: 28, loss: 1.365809e-04
timestep: 29, loss: 1.678807e-04
timestep: 30, loss: 

In [ ]:
vertex_dim = dataset[0].x.shape[1]
edge_dim = dataset[0].edge_attr.shape[1]
hidden_dim = 20
model = THCNet(vertex_dim, edge_dim, hidden_dim).double()

lr = 0.001
lamb = 1e-4

In [ ]:
losses = train(model, dataset, iterations = 200, lr = lr, verbose = verbose, lamb = lamb)